In [1]:
from src.data_structures import Instance
from src.data_structures.features import *
from src.solvers.collection import SolverCollection
import torch
from torch import nn
import os
from pathlib import Path
import sys
torch.set_default_tensor_type(torch.DoubleTensor)
test_instance = Instance.generate(500,0)
solution = SolverCollection.gurobi_optimal(test_instance)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-21


In [2]:
#Training data (Instancias ya solucionadas)
max_size = 1000
training_data_folder = Path("training_data")
instances = map(Instance.from_file,training_data_folder.iterdir()).__iter__()
expand_vector = lambda x,instance: torch.cat([x,torch.zeros(max_size-instance.n_items)])
get_y = lambda instance: expand_vector(torch.tensor(instance.optimal_solution),instance)

In [3]:
#Features
#Debo generar un vector x por cada feature, y cada vector x debe expandirse,
#y luego concatenar todos los vectores x expandidos por lo que la dimension de entrada
#sera #features*n_items.

#Esto se hace por cada instancia

features: list[ItemBatchFeature] = [
    ProfitOverBudget,
    LowerCostOverBudget,
    IsInContSol,
    UpperCostOverBudget
    ]

def gen_x(features: list[ItemBatchFeature],instance: Instance):
    evaluated_features = []
    for feature in features:
        x_feature = torch.tensor(feature.batch_evaluate(instance))
        x_feature = expand_vector(x_feature,instance)
        evaluated_features.append(x_feature)
    return torch.cat(evaluated_features)

In [4]:
class Net(nn.Module):
    def __init__(self, entrada,salida):
        super(Net, self).__init__()

        hidden_size = 50

        self.many = nn.Sequential(
            nn.Linear(entrada, hidden_size),
            nn.Linear(hidden_size,hidden_size),
            nn.Linear(hidden_size,salida),
            nn.Sigmoid()
        )

    def forward(self, x):   
        x = self.many(x)
        return x

In [12]:
net = Net(len(features)*max_size,max_size)
criterion = nn.SmoothL1Loss()
learning_rate = 0.1
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [13]:
#Training
for i in range(4000):
    instance = Instance.generate(np.random.randint(100,1000),10)
    optimizer.zero_grad()
    total_loss = 0
    x = gen_x(features,instance)
    y_pred = net(x)
    batch_loss = criterion(y_pred,get_y(instance) )
    batch_loss.backward()
    optimizer.step()
    total_loss += batch_loss.item()    
    sys.stdout.write(f'\rEpoch {i}, CorrectedLoss {total_loss}')
    sys.stdout.flush()


Epoch 487, CorrectedLoss 0.12029206612976157

KeyboardInterrupt: 

In [14]:
with torch.no_grad():
    test_x = gen_x(features,test_instance)
    test_y = net(test_x)

In [18]:
print(torch.topk(test_y,10,largest=False))
#print(solution.sol)


torch.return_types.topk(
values=tensor([0.3462, 0.3672, 0.3762, 0.3762, 0.3778, 0.3790, 0.3798, 0.3830, 0.3839,
        0.3841]),
indices=tensor([955, 621, 541, 797,  85, 181,  21, 496, 747, 883]))
